In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#import tensorflow as tf
#import keras
from tensorflow.keras.utils import pad_sequences

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
# import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping


2023-09-19 14:37:28.964953: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 14:37:29.090350: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 14:37:29.092874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 14:37:30.096295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def tokenizer(df: pd.DataFrame): 
    data_processed = pd.read_csv("/home/mariannettrd/code/irismarechal1997/moder_ia/data/processed_dataset_v1.csv")
    X = data_processed["text"]
    y = data_processed["offensive"]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)    
    tk = Tokenizer()
    tk.fit_on_texts(X_train) 
    X_tokenized = tk.texts_to_sequences(df)
    return X_tokenized




In [4]:

def padding(X: pd.DataFrame):
     X_train_pad = pad_sequences(X, dtype='float32', padding='post')
     return X_train_pad

In [5]:
def LSTM_model_test(processed=True):

    data_processed = pd.read_csv("/home/mariannettrd/code/irismarechal1997/moder_ia/data/processed_dataset_v1.csv")

    if processed :
        X = data_processed["text"]
        y = data_processed["offensive"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
    else :
        X = data_processed["text_processed"]
        y = data_processed["offensive"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

    ### Let's tokenize the vocabulary
    tk = Tokenizer()
    tk.fit_on_texts(X_train)
    vocab_size = len(tk.word_index)

    # We apply the tokenization to the train and test set
    X_train_token = tokenizer(X_train)
    X_test_token = tokenizer(X_test)

    X_train_pad = padding(X_train_token, dtype='float32', padding='post')
    X_test_pad = padding(X_test_token, dtype='float32', padding='post')

    ### Let's build the neural network now

    # Size of your embedding space = size of the vector representing each word
    embedding_size = 50

    model = Sequential()
    model.add(layers.Embedding(
        input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
        output_dim=embedding_size, # 100
        mask_zero=True, # Built-in masking layer
    ))

    model.add(layers.LSTM(20, return_sequences=True, activation="tanh"))
    model.add(layers.LSTM(20, activation="tanh"))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.summary()

    model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])


    es = EarlyStopping(patience=4, restore_best_weights=True)

    model.fit(X_train_pad, y_train,
            epochs=5,
            batch_size=128,
            callbacks=[es]
            )
    res = model.evaluate(X_test_pad, y_test)
    print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')
    return model